In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/caption/gemini_labeled_28k.csv")

In [3]:
data.head(2)

,image_path,question,answer,candidates,hallucination_candidates,candidates_inx,hallucination_candidates_inx
0,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"['holding', 'playing', 'fencing', 'tennis', 'b...",['bat'],"{'holding': [(40, 47)], 'playing': [(58, 65)],...","{'bat': [(50, 53)]}"
1,2381430.jpg,What do you think is going on in this snapshot?,In this picture we can see one woman is holdin...,"['ground', 'holding', 'playing', 'fencing', 't...","['frisbee', 'bat']","{'ground': [(114, 120)], 'holding': [(40, 47)]...","{'frisbee': [(99, 106)], 'bat': [(50, 53)]}"


In [ ]:
from transformers import AutoTokenizer

def spans_to_token_indices(tokenizer, text, spans):
    """
    Map character-level spans to token index spans after tokenization.
    spans: list of (start_char, end_char) with end exclusive.
    Returns list of (tok_start, tok_end) inclusive in the tokenized sequence (with special tokens).
    """
    enc = tokenizer(
        text,
        add_special_tokens=True,
        return_offsets_mapping=True  # not strictly required for char_to_token, but useful for debug
    )

    results = []
    for (start_char, end_char) in spans:
        # Map to token indices (inclusive)
        tok_start = enc.char_to_token(start_char)
        tok_end   = enc.char_to_token(end_char - 1)

        # If either is None, try to nudge inward/outward a bit (rare, but can happen around spaces)
        if tok_start is None:
            # Move forward until we hit a mappable character or run out
            i = start_char
            while i < end_char and tok_start is None:
                i += 1
                tok_start = enc.char_to_token(i)
        if tok_end is None:
            # Move backward until we hit a mappable character or run out
            i = end_char - 1
            while i >= start_char and tok_end is None:
                i -= 1
                tok_end = enc.char_to_token(i)

        # Final fallback: if still None, the span didn’t map to any token (e.g., only spaces/punct removed)
        if tok_start is None or tok_end is None:
            results.append((None, None))
        else:
            results.append((tok_start, tok_end))
    return enc, results

# # --- Example ---
# # Choose any fast tokenizer; BERT shown here.
# tok = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)



# enc, token_spans = spans_to_token_indices(tok, text, spans)

# print(enc.input_ids)         # includes [CLS] ... [SEP]
# print(token_spans)           # e.g., [(1,1), (4,6), (7,7)] depending on subword splits


/home/user/.conda/envs/shdm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from transformers import AutoTokenizer

# model_path = "liuhaotian/llava-v1.5-7b"
# tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False)

/home/user/.conda/envs/shdm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
sen = data["answer"].iloc[0]

In [11]:
tokenizer.decode(tokenizer(sen)["input_ids"])

'<s> In this picture we can see one woman is holding a bat and playing tennis. Back side, we can see fencing.'

In [14]:
text = "USER: Hello ASSISTANT: How are you ASSISTANT: fine"
token = "ASSISTANT"

# Find the first index of the token
text.find(token)

12

In [18]:
text[:12]

'USER: Hello '

In [15]:

model_name = "liuhaotian/llava-v1.5-7b"

# ✅ Force fast tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, padding_side="left")

/home/user/.conda/envs/shdm/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [16]:
res = tokenizer(text, return_tensors="pt")

In [17]:
res["input_ids"].dtype

torch.int64

In [ ]:
res

In [11]:
import pandas as pd
from collections import Counter

# Sample DataFrame
df = pd.DataFrame({
    "token": ["bat", "holding", "one", "playing", "tennis", "side", "woman", "back", "fencing"],
    "mat_tokens": [[155], [153], [150], [157], [158], [161], [151], [160], [166, 167]],
    "label": [-1, 1, 1, 1, 1, 1, 1, 1, 1]
})

# Step 1: explode mat_tokens so each token ID gets its own row
exploded = df.explode("mat_tokens")

# Step 2: group by mat_token id and compute majority label
def majority_label(labels):
    counts = Counter(labels)
    return counts.most_common(1)[0][0]

mapping = (exploded.groupby("mat_tokens")["label"].apply(majority_label).to_dict())


In [14]:
mapping

{150: 1,
 151: 1,
 153: 1,
 155: -1,
 157: 1,
 158: 1,
 160: 1,
 161: 1,
 166: 1,
 167: 1}

In [1]:
import torch

# Example tensor
t = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
print("Original:", t)

# Position and new values
idx = 3                 # position to replace (value 4)
new_values = torch.tensor([20, 50, 100])

# Create new tensor
result = torch.cat([t[:idx], new_values, t[idx+1:]])
print("Updated:", result)


Original: tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])
Updated: tensor([  1,   2,   3,  20,  50, 100,   5,   6,   7,   8,   9])


Original batch shape: torch.Size([2, 8])
Expanded batch shape: torch.Size([2, 383])


In [3]:
main_tensor = torch.tensor([1, 2, 3, 4, 5])
insert_tensor = torch.tensor([99, 100])
position = 2  # insert before index 2 (i.e., between 2 and 3)

# ✅ Insert using torch.cat
result = torch.cat((main_tensor[:position], insert_tensor, main_tensor[position:]))

In [3]:
import torch
tensor = torch.full((5,), -200)
tensor

tensor([-200, -200, -200, -200, -200])

In [5]:
a = torch.tensor([1, 2, 3, 4, 5])

In [9]:
torch.where(a == 2)[0].tolist()[0]

1

In [28]:
import pandas as pd
df  = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/haloc/embeddings/caption/llava_24/batch_4eec170b-1c8f-4bc1-b4c8-747a7d60b263.jsonl", lines=True, orient='records')